In [1]:
# for natural language processing
import nltk
from nltk.corpus import twitter_samples, stopwords
from nltk.tag import pos_tag
from nltk.stem.wordnet import WordNetLemmatizer
from nltk import FreqDist, classify, NaiveBayesClassifier
from nltk.tokenize import word_tokenize

import re, string
import random

import pickle

In [2]:
positive_tweets = twitter_samples.strings('positive_tweets.json')
negative_tweets = twitter_samples.strings('negative_tweets.json')

In [3]:
pos_tweet_tokens = twitter_samples.tokenized('positive_tweets.json')
neg_tweet_tokens = twitter_samples.tokenized('negative_tweets.json')

In [4]:
# converts each word in tweets to its dictionary form
def lemmatize_sentence(tokens):
    lemmatizer = WordNetLemmatizer()
    lemmatized_sentence = []
    for word, tag in pos_tag(tokens):
        if tag.startswith('NN'):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'
        # stemming with lemmatization needs a part of speech parameter, so that's why we're finding parts of speech
        lemmatized_sentence.append(lemmatizer.lemmatize(word, pos))
    return lemmatized_sentence

In [5]:
# removes extra symbols and extremely common words from tweets
def remove_noise(tweet_tokens, stop_words = ()):
    cleaned_tokens = []

    for token, tag in pos_tag(tweet_tokens):
        token = re.sub('http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+#]|[!*\(\),]|'\
                       '(?:%[0-9a-fA-F][0-9a-fA-F]))+','', token)
        token = re.sub("(@[A-Za-z0-9_]+)","", token)

        if tag.startswith("NN"):
            pos = 'n'
        elif tag.startswith('VB'):
            pos = 'v'
        else:
            pos = 'a'

        lemmatizer = WordNetLemmatizer()
        token = lemmatizer.lemmatize(token, pos)

        if len(token) > 0 and token not in string.punctuation and token.lower() not in stop_words:
            cleaned_tokens.append(token.lower())
    return cleaned_tokens

In [6]:
stop_words = stopwords.words('english')

In [7]:
positive_cleaned_tokens_list = [] # list of lists of positive words
negative_cleaned_tokens_list = [] # list of lists of negatve words

for tokens in pos_tweet_tokens:
    positive_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

for tokens in neg_tweet_tokens:
    negative_cleaned_tokens_list.append(remove_noise(tokens, stop_words))

In [8]:
# currently, tokens are sentences and phrases and words
#     so must break apart each one and collect each individual word
def get_all_words(cleaned_tokens_list):
    for tokens in cleaned_tokens_list:
        for token in tokens:
            yield token # like return, but allows function to resume where it left off, so returns a sequence of values

all_pos_words = get_all_words(positive_cleaned_tokens_list)

In [9]:
freq_dist_pos = FreqDist(all_pos_words)

In [10]:
# prepares lists of lists for positive and negative words for model by formatting
#    them into dictionaries
def get_tweets_for_model(cleaned_tokens_list):
    for tweet_tokens in cleaned_tokens_list:
        yield dict([token, True] for token in tweet_tokens)

positive_tokens_for_model = get_tweets_for_model(positive_cleaned_tokens_list)
negative_tokens_for_model = get_tweets_for_model(negative_cleaned_tokens_list)

In [11]:
positive_dataset = [(tweet_dict, "Lenient on reopening")
                     for tweet_dict in positive_tokens_for_model]

negative_dataset = [(tweet_dict, "Against reopening")
                     for tweet_dict in negative_tokens_for_model]

dataset = positive_dataset + negative_dataset

random.shuffle(dataset)

train_data = dataset[:7000]
test_data = dataset[7000:]

In [12]:
classifier = NaiveBayesClassifier.train(train_data)

print("Accuracy is:", classify.accuracy(classifier, test_data))

print(classifier.show_most_informative_features(10))

Accuracy is: 0.9956666666666667
Most Informative Features
                      :( = True           Agains : Lenien =   2069.5 : 1.0
                      :) = True           Lenien : Agains =    983.4 : 1.0
                     sad = True           Agains : Lenien =     32.6 : 1.0
                follower = True           Lenien : Agains =     21.5 : 1.0
                     bam = True           Lenien : Agains =     20.1 : 1.0
                  arrive = True           Lenien : Agains =     18.4 : 1.0
              appreciate = True           Lenien : Agains =     14.8 : 1.0
               community = True           Lenien : Agains =     14.2 : 1.0
                    poor = True           Agains : Lenien =     13.8 : 1.0
                followed = True           Agains : Lenien =     13.6 : 1.0
None


In [13]:
# testing out how the code would work, but substituted reading in a tweet with a String
custom_tweet = "I ordered just once from TerribleCo, they screwed up, never used the app again."

custom_tokens = remove_noise(word_tokenize(custom_tweet))

print(classifier.classify(dict([token, True] for token in custom_tokens)))

Against reopening


In [14]:
# saving the model built
pickle.dump(classifier, open('tweets_text.sav', 'wb'))